[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/petro1eum/trust_chain/blob/main/examples/trustchain_governance.ipynb)

# TrustChain Governance - Phase 13-15 Demo

This notebook demonstrates:
- **Phase 13**: Policy Layer (YAML rules, deny/allow/require)
- **Phase 14**: Execution Graph (DAG analysis, fork detection)
- **Phase 15**: MCP Security integration

## Setup

In [ ]:
# Install TrustChain (required in Colab)
!uv pip install -q trustchain

In [1]:
from trustchain import ExecutionGraph, PolicyEngine, TrustChain, TrustChainConfig

tc = TrustChain(TrustChainConfig(enable_nonce=False))
print(f"TrustChain ready, key: {tc.get_key_id()[:16]}...")

TrustChain ready, key: 866f71ef-6cc7-41...


## Phase 13: Policy Layer

Runtime policy enforcement for AI governance.

In [2]:
# Define policies in YAML
policy_yaml = """
policies:
  - name: deny_dangerous_tools
    if:
      tool: delete_database
    then:
      deny:
        message: Dangerous operations are blocked

  - name: require_consent_for_pii
    if:
      tool: database_query
      output.contains: ["ssn", "passport", "credit_card"]
    then:
      require:
        - parent_tool: user_consent
"""

engine = PolicyEngine()
engine.load_yaml(policy_yaml)
print(f"Loaded {len(engine.policies)} policies")
for p in engine.policies:
    print(f"  - {p.name}: {p.action.value}")

Loaded 2 policies
  - deny_dangerous_tools: deny
  - require_consent_for_pii: require


In [3]:
# Test 1: Dangerous tool - BLOCKED
dangerous_response = tc._signer.sign("delete_database", {"confirm": True})
passed, violations = engine.evaluate(dangerous_response)
print(f"delete_database: {'ALLOWED' if passed else 'BLOCKED'}")
if violations:
    print(f"  Reason: {violations[0]}")

delete_database: BLOCKED
  Reason: Dangerous operations are blocked


In [4]:
# Test 2: PII access without consent - BLOCKED
pii_response = tc._signer.sign("database_query", {"result": "SSN: 123-45-6789"})
passed, violations = engine.evaluate(pii_response)
print(f"PII query without consent: {'ALLOWED' if passed else 'BLOCKED'}")
if violations:
    print(f"  Reason: {violations[0]}")

PII query without consent: BLOCKED
  Reason: Policy 'require_consent_for_pii': requires parent tool 'user_consent' but no chain provided


In [5]:
# Test 3: PII access WITH consent chain - ALLOWED
consent = tc._signer.sign("user_consent", {"granted": True, "scope": "pii_access"})
pii_with_consent = tc._signer.sign(
    "database_query",
    {"result": "SSN: 123-45-6789"},
    parent_signature=consent.signature
)

passed, violations = engine.evaluate(pii_with_consent, chain=[consent, pii_with_consent])
print(f"PII query WITH consent chain: {'ALLOWED' if passed else 'BLOCKED'}")

PII query WITH consent chain: ALLOWED


## Phase 14: Execution Graph

DAG analysis of agent execution for forensic analysis.

In [6]:
# Simulate a complex agent workflow
responses = []

# Root
start = tc._signer.sign("agent_init", {"session": "abc123"})
responses.append(start)

# Research phase
search = tc._signer.sign("web_search", {"query": "AI safety"}, parent_signature=start.signature)
responses.append(search)

# Fork: two parallel analyses
analyze_a = tc._signer.sign("analyze", {"aspect": "technical"}, parent_signature=search.signature)
analyze_b = tc._signer.sign("analyze", {"aspect": "ethical"}, parent_signature=search.signature)
responses.extend([analyze_a, analyze_b])

# Merge: report from both analyses
report = tc._signer.sign("generate_report", {"sources": 2}, parent_signature=analyze_a.signature)
responses.append(report)

print(f"Agent workflow: {len(responses)} steps")

Agent workflow: 5 steps


In [7]:
# Build execution graph
graph = ExecutionGraph.from_chain(responses)
stats = graph.get_stats()

print("Execution Graph Stats:")
for key, value in stats.items():
    print(f"  {key}: {value}")

Execution Graph Stats:
  total_nodes: 5
  total_roots: 1
  max_depth: 3
  unique_tools: 4
  forks: 1
  replays: 0
  orphans: 0


In [8]:
# Detect forks (where agent branched)
forks = graph.detect_forks()
print(f"\nForks detected: {len(forks)}")
for fork in forks:
    print(f"  At '{fork.parent_tool}': {len(fork.branches)} branches")
    for branch in fork.branches:
        print(f"    -> {branch.tool_id}")


Forks detected: 1
  At 'web_search': 2 branches
    -> analyze
    -> analyze


In [9]:
# Visualize as Mermaid diagram
mermaid = graph.export_mermaid()
print("Mermaid Diagram:")
print("```mermaid")
print(mermaid)
print("```")

Mermaid Diagram:
```mermaid
graph TD
    zkD1YSh4["agent_init"]
    K+ipoRPo["web_search"]
    xTFxdNnz["analyze"]
    RlINh2kH["analyze"]
    nYXGsPP9["generate_report"]
    zkD1YSh4 --> K+ipoRPo
    K+ipoRPo --> xTFxdNnz
    K+ipoRPo --> RlINh2kH
    xTFxdNnz --> nYXGsPP9
    style zkD1YSh4 fill:#90EE90
    style K+ipoRPo fill:#FFD700
```


In [10]:
# Detect potential replay attacks
replay_test = [
    tc._signer.sign("payment", {"amount": 100}),
    tc._signer.sign("payment", {"amount": 100}),
    tc._signer.sign("payment", {"amount": 100}),
]

replay_graph = ExecutionGraph.from_chain(replay_test)
replays = replay_graph.detect_replays()

print(f"Potential Replay Attacks: {len(replays)}")
for r in replays:
    print(f"  Tool '{r.tool_id}': {len(r.occurrences)} identical calls")

Potential Replay Attacks: 1
  Tool 'payment': 3 identical calls


In [11]:
# Get path from root to report
path = graph.get_path(report.signature)
print("\nExecution Path to Report:")
for i, step in enumerate(path):
    print(f"  {i+1}. {step.tool_id}")


Execution Path to Report:
  1. agent_init
  2. web_search
  3. analyze
  4. generate_report


## Combined: Policy + Graph Analysis

Complete AI governance pipeline.

In [12]:
def analyze_agent_session(responses, policy_engine):
    """Complete session analysis with policy and graph checks."""

    # Build graph
    graph = ExecutionGraph.from_chain(responses)

    # Check policies
    policy_violations = []
    for resp in responses:
        passed, violations = policy_engine.evaluate(resp, chain=responses)
        if not passed:
            policy_violations.extend(violations)

    # Check graph anomalies
    forks = graph.detect_forks()
    replays = graph.detect_replays()
    orphans = graph.detect_orphans()

    return {
        "total_operations": len(responses),
        "policy_violations": policy_violations,
        "forks": len(forks),
        "potential_replays": len(replays),
        "orphaned_responses": len(orphans),
        "graph": graph
    }

# Analyze our session
result = analyze_agent_session(responses, engine)

print("Session Analysis:")
print(f"  Total operations: {result['total_operations']}")
print(f"  Policy violations: {len(result['policy_violations'])}")
print(f"  Forks: {result['forks']}")
print(f"  Potential replays: {result['potential_replays']}")
print(f"  Orphans: {result['orphaned_responses']}")
print(f"\nStatus: {'COMPLIANT' if len(result['policy_violations']) == 0 else 'NON-COMPLIANT'}")

Session Analysis:
  Total operations: 5
  Policy violations: 0
  Forks: 1
  Potential replays: 0
  Orphans: 0

Status: COMPLIANT


## Use Cases

- **SOC2/ISO Compliance**: Audit trail + policy enforcement
- **AI Act (EU)**: Runtime governance for AI systems
- **FinTech**: Prevent unauthorized transactions
- **Healthcare (HIPAA)**: PII access control with consent chain
- **Incident Response**: Fork/replay detection for forensics